# UTSS Quickstart

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/obichan117/universal-trading-strategy-schema/blob/main/examples/quickstart.ipynb)

This notebook demonstrates the Universal Trading Strategy Schema (UTSS) - a declarative format for expressing trading strategies that can be validated, shared, and executed by any compliant engine.

## What you'll learn:
1. Define a trading strategy in YAML
2. Validate it against the UTSS schema
3. Run a backtest with real market data
4. Analyze the results

## 1. Installation

Install the packages:

In [ ]:
# Install UTSS packages
!pip install -q utss pyutss

## 2. Define a Strategy

Let's create a simple RSI mean-reversion strategy:
- Buy when RSI(14) drops below 30 (oversold)
- Sell when RSI(14) rises above 70 (overbought)

In [ ]:
strategy_yaml = """
info:
  id: rsi_reversal
  name: RSI Mean Reversion
  version: "1.0"
  description: Buy oversold, sell overbought using RSI(14).
  tags: [reversal, RSI, mean-reversion]

universe:
  type: static
  symbols: [AAPL]

signals:
  rsi_14:
    type: indicator
    indicator: RSI
    params:
      period: 14

conditions:
  oversold:
    type: comparison
    left: { $ref: "#/signals/rsi_14" }
    operator: "<"
    right: { type: constant, value: 30 }
  overbought:
    type: comparison
    left: { $ref: "#/signals/rsi_14" }
    operator: ">"
    right: { type: constant, value: 70 }

rules:
  - name: Buy oversold
    when: { $ref: "#/conditions/oversold" }
    then:
      type: trade
      direction: buy
      sizing: { type: percent_of_equity, percent: 20 }

  - name: Sell overbought
    when: { $ref: "#/conditions/overbought" }
    then:
      type: trade
      direction: sell
      sizing: { type: percent_of_position, percent: 100 }

constraints:
  max_positions: 1
  stop_loss: { percent: 5 }
  no_shorting: true
"""

print("Strategy defined!")

## 3. Validate the Strategy

Use the `utss` package to validate the YAML against the schema:

In [ ]:
from utss import validate_yaml

# Validate and parse the strategy
strategy = validate_yaml(strategy_yaml)

print(f"Strategy: {strategy.info.name}")
print(f"ID: {strategy.info.id}")
print(f"Version: {strategy.info.version}")
print(f"Symbols: {strategy.universe.symbols}")
print(f"Rules: {len(strategy.rules)}")

## 4. Fetch Market Data

Get historical data from Yahoo Finance:

In [ ]:
from datetime import date, timedelta
from pyutss.data import fetch

# Fetch 2 years of AAPL data
end_date = date.today()
start_date = end_date - timedelta(days=730)

data = fetch("AAPL", start_date, end_date)
data.columns = [c.lower() for c in data.columns]  # Normalize column names

print(f"Data shape: {data.shape}")
print(f"Date range: {data.index[0].date()} to {data.index[-1].date()}")
data.tail()

## 5. Run Backtest

Execute the strategy against historical data:

In [ ]:
import yaml
from pyutss import BacktestEngine, BacktestConfig

# Configure the backtest
config = BacktestConfig(
    initial_capital=100000,
    commission_rate=0.001,  # 0.1% per trade
    slippage_rate=0.0005,   # 0.05% slippage
)

# Create engine and run backtest
engine = BacktestEngine(config=config)
strategy_dict = yaml.safe_load(strategy_yaml)

result = engine.run(
    strategy=strategy_dict,
    data=data,
    symbol="AAPL",
)

print(f"Backtest complete!")
print(f"Period: {result.start_date} to {result.end_date}")

## 6. Analyze Results

View the performance metrics:

In [ ]:
print("=" * 50)
print("BACKTEST RESULTS")
print("=" * 50)
print(f"Strategy: {result.strategy_id}")
print(f"Symbol: {result.symbol}")
print(f"Period: {result.start_date} to {result.end_date}")
print()
print("PERFORMANCE")
print("-" * 50)
print(f"Initial Capital: ${result.initial_capital:,.2f}")
print(f"Final Equity:    ${result.final_equity:,.2f}")
print(f"Total Return:    {result.total_return_pct:.2f}%")
print()
print("TRADES")
print("-" * 50)
print(f"Total Trades: {result.num_trades}")

if result.trades:
    winning = sum(1 for t in result.trades if not t.is_open and t.pnl > 0)
    losing = sum(1 for t in result.trades if not t.is_open and t.pnl < 0)
    closed = winning + losing
    if closed > 0:
        print(f"Win Rate:     {winning/closed*100:.1f}%")
        print(f"Winners:      {winning}")
        print(f"Losers:       {losing}")

## 7. View Trade History

In [ ]:
import pandas as pd

if result.trades:
    trades_data = []
    for t in result.trades:
        trades_data.append({
            "Entry Date": t.entry_date,
            "Exit Date": t.exit_date if t.exit_date else "Open",
            "Entry Price": f"${t.entry_price:.2f}",
            "Exit Price": f"${t.exit_price:.2f}" if t.exit_price else "-",
            "Quantity": f"{t.quantity:.1f}",
            "P&L": f"${t.pnl:.2f}" if not t.is_open else "-",
            "P&L %": f"{t.pnl_pct:.2f}%" if not t.is_open else "-",
            "Reason": t.exit_reason if t.exit_reason else "-",
        })
    
    trades_df = pd.DataFrame(trades_data)
    display(trades_df)
else:
    print("No trades executed")

## 8. Plot Equity Curve

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 5))

# Plot equity curve
result.equity_curve.plot(ax=ax, label="Strategy Equity", linewidth=2)

# Add reference line for initial capital
ax.axhline(y=result.initial_capital, color='gray', linestyle='--', alpha=0.5, label='Initial Capital')

ax.set_title(f"{result.strategy_id} - Equity Curve", fontsize=14)
ax.set_xlabel("Date")
ax.set_ylabel("Equity ($)")
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Benchmark Comparison

Compare your strategy's performance against a benchmark (e.g., buy-and-hold SPY):

In [ ]:
from pyutss import calculate_benchmark_metrics
from pyutss.data import fetch
import pandas as pd

# Fetch benchmark data (SPY)
benchmark_data = fetch("SPY", start_date, end_date)
benchmark_data.columns = [c.lower() for c in benchmark_data.columns]

# Calculate daily returns
strategy_returns = result.equity_curve.pct_change().dropna()
benchmark_returns = benchmark_data["close"].pct_change().dropna()

# Normalize indices to date strings for alignment
strategy_returns.index = pd.to_datetime(strategy_returns.index).strftime('%Y-%m-%d')
benchmark_returns.index = benchmark_returns.index.strftime('%Y-%m-%d')

# Calculate benchmark-relative metrics
benchmark_metrics = calculate_benchmark_metrics(strategy_returns, benchmark_returns)

print("=" * 50)
print("BENCHMARK COMPARISON (vs SPY)")
print("=" * 50)
print(f"Alpha (annualized):    {benchmark_metrics.alpha:.2%}")
print(f"Beta:                  {benchmark_metrics.beta:.2f}")
print(f"Information Ratio:     {benchmark_metrics.information_ratio:.2f}")
print(f"Tracking Error:        {benchmark_metrics.tracking_error:.2%}")
print(f"Correlation:           {benchmark_metrics.correlation:.2f}")
print(f"Up Capture Ratio:      {benchmark_metrics.up_capture:.1f}%")
print(f"Down Capture Ratio:    {benchmark_metrics.down_capture:.1f}%")

## 10. Monte Carlo Simulation

Assess strategy robustness by shuffling trade order and computing confidence intervals:

In [ ]:
from pyutss import MonteCarloSimulator

# Only run if we have closed trades
closed_trades = [t for t in result.trades if not t.is_open] if result.trades else []

if len(closed_trades) >= 3:
    # Convert trades to format for Monte Carlo
    trades_for_mc = [{"pnl": t.pnl} for t in closed_trades]
    
    # Run Monte Carlo simulation
    simulator = MonteCarloSimulator(seed=42)
    mc_result = simulator.shuffle_trades(
        trades_for_mc,
        initial_capital=result.initial_capital,
        n_iterations=1000
    )
    
    print("=" * 50)
    print("MONTE CARLO SIMULATION (1,000 iterations)")
    print("=" * 50)
    print(f"Max Drawdown (95th percentile): {mc_result.drawdown_95:.2%}")
    print(f"Max Drawdown (99th percentile): {mc_result.drawdown_99:.2%}")
    print(f"Return 95% CI: [{mc_result.return_ci[0]:.2%}, {mc_result.return_ci[1]:.2%}]")
    print(f"Sharpe 95% CI: [{mc_result.sharpe_ci[0]:.2f}, {mc_result.sharpe_ci[1]:.2f}]")
else:
    print("Need at least 3 closed trades for Monte Carlo simulation")

## 11. Try Different Strategies

Here are more strategy patterns you can try:

### Golden Cross (Trend Following)

In [ ]:
golden_cross_yaml = """
info:
  id: golden_cross
  name: Golden Cross Strategy
  version: "1.0"

universe:
  type: static
  symbols: [SPY]

rules:
  - name: Golden Cross Entry
    when:
      type: expr
      formula: "SMA(50)[-1] <= SMA(200)[-1] and SMA(50) > SMA(200)"
    then:
      type: trade
      direction: buy
      sizing: { type: percent_of_equity, percent: 100 }

  - name: Death Cross Exit
    when:
      type: expr
      formula: "SMA(50)[-1] >= SMA(200)[-1] and SMA(50) < SMA(200)"
    then:
      type: trade
      direction: sell
      sizing: { type: percent_of_position, percent: 100 }

constraints:
  trailing_stop: { percent: 10 }
"""

# Validate it
gc_strategy = validate_yaml(golden_cross_yaml)
print(f"Validated: {gc_strategy.info.name}")
print("Uses expr formulas for crossover detection!")

### Calendar Strategy (Buy Monday, Sell Friday)

In [ ]:
calendar_yaml = """
info:
  id: weekly_momentum
  name: Weekly Momentum
  version: "1.0"

universe:
  type: static
  symbols: [SPY]

signals:
  day_of_week:
    type: calendar
    field: day_of_week

rules:
  - name: Monday Entry
    when:
      type: comparison
      left: { $ref: "#/signals/day_of_week" }
      operator: "="
      right: { type: constant, value: 0 }  # Monday
    then:
      type: trade
      direction: buy
      sizing: { type: percent_of_equity, percent: 50 }

  - name: Friday Exit
    when:
      type: comparison
      left: { $ref: "#/signals/day_of_week" }
      operator: "="
      right: { type: constant, value: 4 }  # Friday
    then:
      type: trade
      direction: sell
      sizing: { type: percent_of_position, percent: 100 }

constraints:
  no_shorting: true
"""

cal_strategy = validate_yaml(calendar_yaml)
print(f"Validated: {cal_strategy.info.name}")
print("Uses calendar signals for day-of-week trading!")

## Next Steps

- **Documentation**: https://obichan117.github.io/utss/
- **Examples**: See `examples/` directory for more strategies
- **Schema Reference**: Check `docs/specification.md` for full schema details

### Available Packages

| Package | Description |
|---------|-------------|
| `utss` | Schema validation and Pydantic models |
| `pyutss` | Backtesting engine with data providers |
| `utss-llm` | Natural language to UTSS conversion |
| `utss-mcp` | MCP server for Claude Code integration |